In [11]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from PIL import Image
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
imgs_path = "Train"
data_list = []
labels_list = []
classes_list = 43 
for i in range(classes_list):
    i_path = os.path.join(imgs_path, str(i)) 
    for img in os.listdir(i_path):
        im = Image.open(i_path +'/'+ img)
        im = im.resize((32,32))
        im = np.array(im)
        data_list.append(im)
        labels_list.append(i)
data = np.array(data_list)
labels = np.array(labels_list)

In [3]:
def prep_dataset(X,y):
    X_prep = X.astype('float32')
    y_prep = to_categorical(np.array(y))
    return (X_prep, y_prep)
X, y = prep_dataset(data,labels)

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, shuffle=True,stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val,y_val, test_size=0.5, shuffle=True)

In [5]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',input_shape=X.shape[1:]))
model.add(layers.MaxPool2D(strides=2))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(filters=128,kernel_size=3,activation='relu',padding='same'))
model.add(layers.MaxPool2D(strides=2))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(filters=512,kernel_size=3,activation='relu',padding='same'))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(filters=512,kernel_size=3,activation='relu',padding='same'))
model.add(layers.MaxPool2D(strides=2))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())

model.add(layers.Dense(4000,activation='relu'))
model.add(layers.Dense(4000,activation='relu'))
model.add(layers.Dense(1000,activation='relu'))
model.add(layers.Dense(43,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 32)       128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0

In [6]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history= model.fit(X_train,y_train,epochs=10,batch_size=64,validation_data=(X_val,y_val))

Epoch 1/10
491/491 [==============================] - 297s 602ms/step - loss: 1.7172 - accuracy: 0.5119 - val_loss: 0.3964 - val_accuracy: 0.8567
Epoch 2/10
491/491 [==============================] - 304s 620ms/step - loss: 0.2363 - accuracy: 0.9247 - val_loss: 0.1436 - val_accuracy: 0.9490
Epoch 3/10
491/491 [==============================] - 306s 624ms/step - loss: 0.1137 - accuracy: 0.9640 - val_loss: 0.0503 - val_accuracy: 0.9867
Epoch 4/10
491/491 [==============================] - 307s 626ms/step - loss: 0.0977 - accuracy: 0.9714 - val_loss: 0.0546 - val_accuracy: 0.9842
Epoch 5/10
491/491 [==============================] - 289s 589ms/step - loss: 0.0805 - accuracy: 0.9763 - val_loss: 0.0837 - val_accuracy: 0.9750
Epoch 6/10
491/491 [==============================] - 282s 575ms/step - loss: 0.0677 - accuracy: 0.9827 - val_loss: 0.0527 - val_accuracy: 0.9883
Epoch 7/10
491/491 [==============================] - 279s 568ms/step - loss: 0.0612 - accuracy: 0.9832 - val_loss: 0.0556 -

In [7]:
y_test = np.argmax(y_test,axis=1)
y_pred= model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

123/123 [==============================] - 8s 66ms/step


In [9]:
print('-train acuracy: {:.2f}%\n-accuracy: {:.2f}%\n-val_accuracy: {:.2f}%'.format(accuracy_score(y_test,y_pred)*100,(history.history['accuracy'][-1])*100,(history.history['val_accuracy'][-1])*100))

-train acuracy: 99.26%
-accuracy: 98.94%
-val_accuracy: 99.36%


In [14]:
y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

with tf.device('/GPU:0'):
    for img in imgs:
        image = Image.open('../Capstone Project/'+img)
        image = image.resize([32, 32])
        data.append(np.array(image))

X_test=np.array(data)

with tf.device('/GPU:0'):
    pred = np.argmax(model.predict(X_test), axis=-1)

from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

395/395 [==============================] - 26s 64ms/step
0.9539192399049882
